# Zero shot classification

## [DeBerta-v3-base-mnli-fever-anli](https://huggingface.co/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli?candidateLabels=covid-19%2C+vaccine+efficacy%2C+vaccine+side+effects%2C+measles%2C+cholera%2C+home+remedies%2C+conspiracy&multiClass=true&text=actor+jamie+foxx+reportedly+par+lyzed+and+bl+nd+due+to+covid+19+vaccine+american+actor+jamie+foxx+has+reportedly+s+ffered+partial+par+lysis+following+a+covid+19+vaccine+complication+the+vaccine+according+to+hollywood+journalist+a+j+benza+resulted+in+a+bl+d+clot+in+his+brain+that+led+to+him+being+partially+par+lyzed+and+bl+nd+in+addition+to+a+series+of+other+complications+benza+claims+the+news+was+confirmed+by+a+source+close+to+foxx+jamie+had+a+bl+d+clot+in+his+brain+after+he+got+the+sh+t+he+did+not+want+the+sh+t+but+the+movie+he+was+on+he+was+pressured+to+get+it+confessed+the+journalist+who+formally+worked+as+a+columnist+for+the+new+york+daily+news+during+an+appearance+on+dr+drew+pinsky+s+online+show+ask+dr+drew+the+bl+d+clot+in+the+brain+caused+him+at+that+point+to+be+partially+par+lyzed+and+bl+nd+benza+alleged+as+he+insisted+his+insider+was+someone+in+the+room+with+first+hand+knowledge+of+foxx+s+hospitalization+foxx+was+admitted+to+the+hospital+after+s+ffering+a+medical+complication+on+thursday+april+11+according+to+his+daughter+corinne+foxx+luckily+due+to+quick+action+and+great+care+he+is+already+on+his+way+to+recovery+we+know+how+beloved+he+is+and+appreciate+your+prayers+the+family+asks+for+privacy+during+this+time+she+said+the+55+year+old+s+medical+woes+began+while+filming+his+latest+movie+back+in+action+the+movie+also+starring+cameron+diaz+is+said+to+have+moved+forward+with+production+without+foxx+using+a+body+double+in+his+place)

In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

2023-07-21 18:21:16.359443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

2023-07-21 18:21:19.202167: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-21 18:21:19.203406: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-21 18:21:19.229901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-21 18:21:19.230761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-07-21 18:21:19.230804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-07-21 18:21:19.231159: W tensorflow/stream_executor/platform/defau

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/417 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

### Example

In [6]:
sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = ["hello kitty", "dog", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=True)
print(output)


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU', 'labels': ['hello kitty', 'environment', 'dog', 'entertainment'], 'scores': [0.5422002673149109, 0.5266703367233276, 0.3002735674381256, 0.02254047431051731]}


### Read in data

In [7]:
# read in data export
df = pd.read_csv("amp.csv")

In [8]:
# set labels for themes
theme_labels = df['themeName'].unique()

# set labels for issues
issue_labels = df['issueName'].unique()

In [9]:
theme_labels

array(['Vaccine Side Effects', 'Home Remedies', nan, 'Corruption',
       'Conspiracy', 'Vaccine Efficacy', 'Case Reporting',
       'Traditional/Religious Practices', 'Bio-weapon', 'Variants',
       'Treatment', 'Media Bias', 'Stigmatization', 'Disease Risk'],
      dtype=object)

## Write function to classify text and return a df

In [10]:
def classify_themes(df):
    result_list = []
    for index, row in df.iterrows():
        text_sequence = row['text']
        result = classifier(text_sequence, theme_labels, multi_label = True)
        result['predictedThemes'] = result['labels'][0:3]
        result['themeScores'] = result['scores'][0:3]
        result_list.append(result)
    result_df = pd.DataFrame(result_list)[['sequence', 'predictedThemes', 'themeScores']]
    return result_df

In [11]:
def classify_issues(df):
    result_list = []
    for index, row in df.iterrows():
        text_sequence = row['text']
        result = classifier(text_sequence, issue_labels, multi_label = False)
        result['predictedIssues'] = result['labels'][0]
        result['issueScores'] = result['scores'][0]
        result_list.append(result)
    result_df = pd.DataFrame(result_list)[['sequence', 'predictedIssues', 'issueScores']]
    return result_df

In [12]:
df.columns

Index(['date', 'attachments', 'textTranslated', 'detectedLanguage',
       'likeCount', 'countries', 'authorId', 'url', 'platform', 'commentCount',
       'socialMediaPostType', 'shareCount', 'createdAt', 'submittedLanguage',
       'themeIds', 'issueIds', 'text', 'updatedAt', 'id', 'index', 'type',
       'issueName', 'themeName'],
      dtype='object')

In [178]:
# predict themes
themes_df = classify_themes(df)

In [180]:
# predict issues
issues_df = classify_issues(df)

In [182]:
# combine results
results_df =  issues_df.merge(themes_df, how = "left", on = ["sequence"])

In [184]:
# write to csv
results_df.to_csv("predicted_amp.csv", index = False)

## Combine to original dataframe

In [188]:
df = df.merge(results_df, how = "left", left_on = "text", right_on = "sequence")

In [191]:
df.head()

,date,attachments,textTranslated,detectedLanguage,likeCount,countries,authorId,url,platform,commentCount,socialMediaPostType,shareCount,createdAt,submittedLanguage,themeIds,issueIds,text,updatedAt,id,index,type,issueName,themeName,predicted_labels,sequence,predictedIssues,issueScores,predictedThemes,themeScores
0,2023-06-03,2023/06/08/evidence/749a5972-c537-4f93-80de-ad...,[object Object],en,76.0,NG,rhysoneill@gmail.com,https://www.facebook.com/instablog9ja/posts/pf...,Facebook,9.0,Post,4.0,2023-06-08T16:14:34.393Z,en,e83rfYgBnMmOXZCbaYt2,g80Cf4gBnMmOXZCbX4tn,actor jamie foxx reportedly par lyzed and bl n...,2023-06-08T16:14:34.393Z,749a5972-c537-4f93-80de-adb0c73527a2,prod-evidence-v1,_doc,COVID-19,Vaccine Side Effects,NaN,actor jamie foxx reportedly par lyzed and bl n...,COVID-19,0.882953,"[Vaccine Side Effects, Vaccine Efficacy, Case ...","[0.9748157262802124, 0.9450281858444214, 0.891..."
1,2023-06-05,NaN,[object Object],en,2.0,KE,rhysoneill@gmail.com,https://www.facebook.com/robertalai/posts/pfbi...,Facebook,NaN,Comment,NaN,2023-06-08T17:13:15.540Z,en,fs31fogBnMmOXZCbnosl,g80Cf4gBnMmOXZCbX4tn,use alot of fresh ginger when it strikes your ...,2023-06-08T17:13:15.540Z,11b135fd-d34f-4bc5-9e51-b4ed28f22a22,prod-evidence-v1,_doc,COVID-19,Home Remedies,NaN,use alot of fresh ginger when it strikes your ...,Measles,0.203130,"[Home Remedies, Treatment, Vaccine Efficacy]","[0.8939719796180725, 0.8867986798286438, 0.683..."
2,2023-06-07,NaN,[object Object],en,1.0,ZA,rhysoneill@gmail.com,https://www.facebook.com/permalink.php?story_f...,Facebook,NaN,Comment,NaN,2023-06-08T18:24:00.164Z,en,NaN,g80Cf4gBnMmOXZCbX4tn,vaccine kills government pushed a unapproved v...,2023-06-08T18:24:00.164Z,9f635c18-49c4-4d26-ada9-5d6ae2706f1c,prod-evidence-v1,_doc,COVID-19,NaN,NaN,vaccine kills government pushed a unapproved v...,Polio,0.218444,"[Vaccine Side Effects, Corruption, Conspiracy]","[0.9939225316047668, 0.9603886008262634, 0.952..."
3,2023-05-17,2023/06/08/evidence/af0d9542-930b-4101-a786-20...,[object Object],en,NaN,"ZA,Africa",rhysoneill@gmail.com,https://t.me/SAAwakened/138313,Telegram,NaN,NaN,NaN,2023-06-08T14:30:29.975Z,en,es3rfYgBnMmOXZCbaYt2,g80Cf4gBnMmOXZCbX4tn,reminder share share share stop medical col...,2023-06-08T14:30:29.975Z,af0d9542-930b-4101-a786-202710a13f26,prod-evidence-v1,_doc,COVID-19,Corruption,NaN,reminder share share share stop medical col...,Cholera,0.774956,"[Disease Risk, Treatment, Corruption]","[0.9749229550361633, 0.746890664100647, 0.7359..."
4,2023-06-08,2023/06/08/evidence/085206d6-bbb6-460e-b96e-98...,[object Object],en,NaN,ZA,rhysoneill@gmail.com,NaN,WhatsApp,NaN,NaN,NaN,2023-06-08T14:19:09.429Z,en,e83rfYgBnMmOXZCbaYt2,NaN,the south african government has established a...,2023-06-08T14:19:09.429Z,ea9ee102-2d4f-48f3-b1f4-13b2cb4714ba,prod-evidence-v1,_doc,NaN,Vaccine Side Effects,NaN,the south african government has established a...,COVID-19,0.956345,"[Vaccine Side Effects, nan, Vaccine Efficacy]","[0.9132343530654907, 0.4491819143295288, 0.406..."


In [193]:
mismatch = df[df["issueName"] != df["predictedIssues"]]

In [195]:
mismatch.shape

(38, 29)

In [201]:
mismatch.dropna(subset = ["issueName"], inplace = True)

/tmp/ipykernel_546/2144932955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatch.dropna(subset = ["issueName"], inplace = True)


In [203]:
mismatch[["text", "issueName", "predictedIssues"]]

,text,issueName,predictedIssues
1,use alot of fresh ginger when it strikes your ...,COVID-19,Measles
2,vaccine kills government pushed a unapproved v...,COVID-19,Polio
3,reminder share share share stop medical col...,COVID-19,Cholera
7,des bébés allaités par des mères qui ont reçu ...,COVID-19,NaN
16,the difference between europe and africa,COVID-19,NaN
18,a sense of concern has taken over the fans as ...,COVID-19,NaN
19,so they want to bring down museveni for opposi...,COVID-19,NaN
39,healing for various illnesses and diseases of ...,Cholera,NaN
40,yes ooo is all about mera herbal cleanser cont...,Malaria,NaN
41,the us now wants to finish him because he reje...,COVID-19,NaN


In [204]:
mismatch.to_csv("mismatch_issues.csv", index = False)